<a href="https://colab.research.google.com/github/asya474/semantic_segmentation/blob/main/%5Bhw%5Dsemantic_segmentation_usual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!wget -c /content/drive/MyDrive/Colab_Notebooks/PH2Dataset.rar

/content/drive/MyDrive/Colab_Notebooks/PH2Dataset.rar: Scheme missing.


In [3]:
!ls

drive  sample_data


In [ ]:
!pwd

In [ ]:
images=[]
lesions=[]
from skimage.io import imread
import os

root='kaggle\input\hw5-rodinki/PH2Dataset'

for root, dirs, files in os.walk(os.path.join(root, 'PH2 Dataset Images')):
  if root.endswith('_Dermoscopic_Image'):
    images.append(imread(os.path.join(root, files[0])))
  if root.endswith('_lesion'):
    lesions.append(imread(os.path.join(root, files[0])))

In [ ]:
from skimage.transform import resize
size=(256, 256)
X=[resize (x, size, mode='constant', anti_aliasing=True,) for x in images]
Y=[resize (y, size, mode='constant', anti_aliasing=False,)> 0.5 for y in lesions ]

In [ ]:
import numpy as np
X=np.array(X, np.float32)
Y=np.array(Y, np.float32)
print(f'loaded {len(X)} images')

In [ ]:
len(lesions)

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output

plt.figure(fig=(18, 6))
for i in range(6):
  plt.subplot(2, 6, i+1)
  plt.axis("off")
  plt.imshow(X[i])

  plt.subplot(2, 6, i+7)
  plt.axis('off')
  plt.imshow(Y[i])
plt.show();

In [ ]:
ix=np.random.choice(len(X), len(X), False)
tr, val, ts=np.split(ix, [100, 150])

In [ ]:
print(len(tr), len(val), len(ts))

In [ ]:
from torch.utils.data import DataLoader
batch_size=25
data_tr=DataLoader(list(zip(np.rollaxis(X[tr], 3, 1), Y[tr, np.newaxis])),
                   batch_size=batch_size, shuffle=True)
data_val=DataLoader(list(zip(np.rollaxis(X[val], 3, 1), Y[val, np.newaxis])),
                   batch_size=batch_size, shuffle=True)
data_ts=DataLoader(list(zip(np.rollaxis(X[ts], 3, 1), Y[ts, np.newaxis])),
                   batch_size=batch_size, shuffle=True)

In [ ]:
import torch
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.Functional as F
from torchvision import models
import torch.optim as optim
from time import time

from matplotlib import rcParams
rcParams['figure.figsize'] = (15, 6)

In [ ]:
class SegNet(nn.Module):
  def __init__(self):
    super().__init__()

    self.enc_conv0=nn.Sequential(nn.BatchNorm2d(num_features=3),
                                 nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
                                 nn.ReLU())
    self.pool0=nn.MaxPool2d(2, return_indices=True)

    self.enc_conv1=nn.Sequential(nn.BatchNorm2d(num_features=64),
                                 nn.Conv2d(in_cnannels=64, out_channels=128, kernel_size=3, padding=1),
                                 nn.ReLU())
    self.pool1=nn.MaxPool2d(2, return_indices=True)

    self.enc_conv2=nn.Sequential(nn.BatchNorm2d(num_features=128),
                                 nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
                                 nn.ReLU())
    self.pool2=nn.MaxPool2d(2, return_indices=True)

    self.enc_conv3=nn.Sequential(nn.BatchNorm2d(num_features=256),
                                 nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
                                 nn.ReLU())
    self.pool3=nn.MaxPool2d(2, return_indices=True)

    #bottleneck
    self.bottleneck_conv=nn.Sequential(nn.BatchNorm2d(num_features=512),
                                       nn.Conv2d(in_channels=512, out_channels=512, kernel_size=1),
                                       nn.ReLU())
    
    #decoder(upsampling)
    self.upsample0=nn.MaxUnpool2d(2)
    self.dec_conv0=nn.Sequential(nn.BatchNorm2d(num_features=512),
                                 nn.Conv2d(in_channels=512, out_channels=256, kernel_size=3, padding=1),
                                 nn.ReLU())
    
    self.upsample1=nn.MaxUnpool2d(2)
    self.dec_conv1=nn.Sequential(nn.BatchNorm2d(num_features=256),
                                 nn.Conv2d(in_channels=256, out_channels=128, kernel_size=3, padding=1),
                                 nn.ReLU())
    
    self.upsample2=nn.MaxUnpool2d(2)
    self.dec_conv2=nn.Sequential(nn.BatchNorm2d(num_features=128),
                                 nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, padding=1),
                                 nn.ReLU())
    
    self.upsample3=nn.MaxUnpool2d()
    self.dec_conv3=nn.Sequential(nn.BatchNorm2d(num_features=64),
                                 nn.Conv2d(in_channels=64, out_channels=1, kernel_size=3, padding=1))
    
  def forward(self, x):
    e0, indices3=self.pool0(self.enc_conv0(x))  
    e1, indices2=self.pool1(self.enc_conv1(e0))
    e2, indices1=self.pool2(self.enc_conv2(e1))
    e3, indices0=self.pool3(self.enc_conv3(e2))
    b=self.bottleneck_conv(e3)
    d0=self.dec_conv0(self.upsample0(b, indices0))
    d1=self.dec_conv1(self.upsample1(d0, indices1))
    d2=self.dec_conv2(self.upsample2(d1, indices2))
    d3=self.dec_conv3(self.upsample2(d2, indices3))
    return d3
